In [14]:
import google.generativeai as genai
import os
from selenium import webdriver
from selenium.webdriver.chrome.options import Options
from selenium.webdriver.chrome.service import Service
from selenium.webdriver.common.by import By 

In [44]:
class Scraper:
     def scrapingData(self, url):
        DRIVER_PATH = 'C:/path/to/chromedriver/chromedriver-win64/chromedriver.exe'  
        options = Options()
        options.add_argument('--headless')
        service = Service(executable_path=DRIVER_PATH)  
        driver = webdriver.Chrome(service=service, options=options)
        driver.get(url)
        
        elements = driver.find_elements(By.XPATH, "//h2 | //p | //ul | //i")
        content = []

        for element in elements:
            text = element.text.strip() 
            if not text: 
                continue
            if element.tag_name == 'ul':
                li_elements = element.find_elements(By.TAG_NAME, 'li')
                list_items = [li.text.strip() for li in li_elements if li.text.strip()]
                if list_items:
                    content.append(" ".join(list_items))  
            else:
                content.append(text)  
        driver.quit()
        return content
        

In [45]:
url = 'https://www.presight.io/privacy-policy.html'
scraper = Scraper()
content = scraper.scrapingData(url)
content

['PRIVACY POLICY',
 'Last updated 15 Sep 2023',
 'At Presight, we are committed to protecting the privacy of our customers and visitors to our website. This Privacy Policy explains how we collect, use, and disclose information about our customers and visitors.',
 'Information Collection and Use',
 'We collect several different types of information for various purposes to provide and improve our Service to you.',
 'Types of Data Collected',
 'Personal Data',
 'While using our Service, we may ask you to provide us with certain personally identifiable information that can be used to contact or identify you ("Personal Data"). Personally identifiable information may include, but is not limited to:',
 'Email address First name and last name Phone number Address, State, Province, ZIP/Postal code, City Cookies and Usage Data',
 'Usage Data',
 'We may also collect information that your browser sends whenever you visit our Service or when you access the Service by or through a mobile device ("Us